<a href="https://colab.research.google.com/github/alrahimi/AGI/blob/main/AGI_Reasoning_Pipeline_R08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AGI Reasoning Pipeline — R08
**Focus:** Proof tracing + narrative explanation  
**Date:** 2025-10-30  
**Lineage:** R01→R02→R03→R04→R07→**R08**  

This version adds a *human-facing* proof layer on top of the stable R07 logic pipeline:

- keep: Premises vs Goals discipline  
- keep: NL → Logic → Z3  
- keep: “No X are Y”, “Some X are Y”, plural→singular  
- **add:** step-by-step *reasoning narrative*  
- **add:** “why not entailed” phrasing that sounds like an agent explaining itself


## 1. Overview

R08 is a *presentation* upgrade: we already know how to check entailment, but we want to **show the proof**.

We do this by:
1. Capturing all translated premises.
2. Capturing each goal in logic form.
3. Emitting a “pseudo-proof” script that shows how Z3 *could* have concluded the goal (for simple cases).
4. Falling back to the R07 diagnostics when the goal is **not** entailed.

This is intentionally simple and transparent so it can be embedded in an agent later.


In [ ]:
# 2. Setup
!pip -q install z3-solver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.3/29.3 MB 15.6 MB/s eta 0:00:00


In [ ]:
# 3. Imports & globals
import re
from z3 import *

U = DeclareSort('U')
PRED_CACHE = {}

def get_pred(name, arity=1):
    key = (name, arity)
    if key not in PRED_CACHE:
        dom = [U] * arity
        PRED_CACHE[key] = Function(name, *(dom + [BoolSort()]))
    return PRED_CACHE[key]

In [ ]:
# 4. Canonicalization

CANONICAL_NOUNS = {
    "men": "man",
    "people": "person",
    "doctors": "doctor",
    "surgeons": "surgeon",
    "students": "student",
    "cats": "cat",
    "dogs": "dog",
}

def normalize_const(name: str) -> str:
    return re.sub(r'[^a-z0-9_]', '', name.strip().lower())

def noun_to_pred(n: str) -> str:
    n = n.strip().lower()
    if n in CANONICAL_NOUNS:
        return CANONICAL_NOUNS[n]
    if n.endswith("ies") and len(n) > 3:
        n = n[:-3] + "y"
    elif n.endswith("s") and len(n) > 3:
        n = n[:-1]
    return re.sub(r'[^a-z0-9_]', '_', n)

def verb_phrase_to_pred(vp: str) -> str:
    return re.sub(r'[^a-z0-9_]', '_', vp.strip().lower())

def sentence_to_const(name, consts):
    c = normalize_const(name)
    if c not in consts:
        consts[c] = Const(c, U)
    return consts[c]

In [ ]:
# 5. English → Logic translator (same core as R07)

ALL_X_ARE_Y    = re.compile(r'^(all|every)\s+([a-z ]+?)\s+are\s+([a-z ]+)$', re.I)
NO_X_ARE_Y     = re.compile(r'^no\s+([a-z ]+?)\s+are\s+([a-z ]+)$', re.I)
SOME_X_ARE_Y   = re.compile(r'^(some|there is a|there is an)\s+([a-z ]+?)\s+are?\s+([a-z ]+)$', re.I)
A_IS_A_X       = re.compile(r'^([A-Z][a-z0-9_]*)\s+is\s+(?:an?\s+)?([a-z ]+)$')
A_IS_NOT_X     = re.compile(r'^([A-Z][a-z0-9_]*)\s+is\s+not\s+([a-z ]+)$', re.I)
ANY_X_CAN_Y    = re.compile(r'^(any(one)?|whoever|who)\s+(.+?)\s+can\s+(.+)$', re.I)

def translate_sentence_to_z3(s: str, consts: dict):
    s = s.strip().rstrip('.')

    m = ALL_X_ARE_Y.match(s)
    if m:
        X = noun_to_pred(m.group(2))
        Y = noun_to_pred(m.group(3))
        x = Const('x', U)
        fol = ForAll([x], Implies(get_pred(X)(x), get_pred(Y)(x)))
        return {"raw": s, "kind": "forall_pos", "fol": fol, "pretty": f"∀x: {X}(x) → {Y}(x)", "preds": {X, Y}}

    m = NO_X_ARE_Y.match(s)
    if m:
        X = noun_to_pred(m.group(1))
        Y = noun_to_pred(m.group(2))
        x = Const('x', U)
        fol = ForAll([x], Implies(get_pred(X)(x), Not(get_pred(Y)(x))))
        return {"raw": s, "kind": "forall_neg", "fol": fol, "pretty": f"∀x: {X}(x) → ¬{Y}(x)", "preds": {X, Y}}

    m = SOME_X_ARE_Y.match(s)
    if m:
        X = noun_to_pred(m.group(2))
        Y = noun_to_pred(m.group(3))
        x = Const('x', U)
        fol = Exists([x], And(get_pred(X)(x), get_pred(Y)(x)))
        return {"raw": s, "kind": "exists", "fol": fol, "pretty": f"∃x: {X}(x) ∧ {Y}(x)", "preds": {X, Y}}

    m = ANY_X_CAN_Y.match(s)
    if m:
        rest = m.group(3).strip()
        if rest.startswith('is '):
            rest = rest[3:]
        X = noun_to_pred(rest)
        Y = verb_phrase_to_pred(m.group(4))
        x = Const('x', U)
        fol = ForAll([x], Implies(get_pred(X)(x), get_pred(Y)(x)))
        return {"raw": s, "kind": "forall_can", "fol": fol, "pretty": f"∀x: {X}(x) → {Y}(x)", "preds": {X, Y}}

    m = A_IS_NOT_X.match(s)
    if m:
        a = m.group(1)
        X = noun_to_pred(m.group(2))
        aC = sentence_to_const(a, consts)
        fol = Not(get_pred(X)(aC))
        return {"raw": s, "kind": "fact_neg", "fol": fol, "pretty": f"¬{X}({aC})", "preds": {X}}

    m = A_IS_A_X.match(s)
    if m:
        a = m.group(1)
        X = noun_to_pred(m.group(2))
        aC = sentence_to_const(a, consts)
        fol = get_pred(X)(aC)
        return {"raw": s, "kind": "fact", "fol": fol, "pretty": f"{X}({aC})", "preds": {X}}

    return {"raw": s, "kind": "unknown", "fol": None, "pretty": f"Unparsed: {s}", "preds": set()}

In [ ]:
# 6. Input parser (Premises/Goals, multi-goal)

def split_sentences(text: str):
    parts = re.split(r'\.\s+|\;\s+|\n', text)
    return [p.strip() for p in parts if p.strip()]

def parse_input_block(text: str):
    premises, goals = [], []
    for line in text.split('\n'):
        line = line.strip()
        if not line:
            continue
        lower = line.lower()
        if lower.startswith('premise:') or lower.startswith('premises:'):
            rest = line.split(':', 1)[1]
            premises.extend(split_sentences(rest))
        elif lower.startswith('goal:') or lower.startswith('goals:'):
            rest = line.split(':', 1)[1]
            goals.extend(split_sentences(rest))
        else:
            premises.append(line)
    return premises, goals

In [ ]:
# 7. Goal → Z3 with metadata

def goal_to_z3(s: str, consts: dict):
    s = s.strip().rstrip('.')

    m = re.match(r'^([A-Z][a-z0-9_]*)\s+is\s+(?:an?\s+)?([a-z ]+)$', s)
    if m:
        a = m.group(1)
        X = noun_to_pred(m.group(2))
        aC = sentence_to_const(a, consts)
        return get_pred(X)(aC), f"{X}({aC})", {X}, {aC}

    m = re.match(r'^([A-Z][a-z0-9_]*)\s+can\s+(.+)$', s, re.I)
    if m:
        a = m.group(1)
        Y = verb_phrase_to_pred(m.group(2))
        aC = sentence_to_const(a, consts)
        return get_pred(Y)(aC), f"{Y}({aC})", {Y}, {aC}

    toks = s.split()
    if len(toks) == 2 and re.match(r'^[A-Z]', toks[0]):
        a = toks[0]
        X = noun_to_pred(toks[1])
        aC = sentence_to_const(a, consts)
        return get_pred(X)(aC), f"{X}({aC})", {X}, {aC}

    return None, f"Unparsed goal: {s}", set(), set()

In [ ]:
# 8. Entailment core

def check_entailment(premises, goal):
    s = Solver()
    for p in premises:
        s.add(p)
    s.push()
    s.add(Not(goal))
    res = s.check()
    if res == unsat:
        s.pop()
        return True, None
    else:
        model = s.model()
        s.pop()
        return False, model

In [ ]:
# 9. Proof narrator
# We can't always get a full Z3 proof tree here, so we build a
# HUMAN-FACING chain from the premises we *do* have.

def build_proof_narrative(goal_pretty, premises_translated):
    # Look for pattern: ∀x: A(x) → B(x)  AND  A(const)  ⇒  B(const)
    # This covers the classic "All men are mortal. Alex is a man. ⇒ Alex is mortal."
    goal_pred = None
    goal_const = None
    m = re.match(r'^([a-z_]+)\((.+)\)$', goal_pretty)
    if m:
        goal_pred = m.group(1)
        goal_const = m.group(2)

    steps = []
    if goal_pred is None:
        steps.append("Could not parse goal structure for narrative.")
        return steps

    # find rules that conclude goal_pred
    rules = []
    facts = []
    for tr in premises_translated:
        ptxt = tr["pretty"]
        # rule: ∀x: A(x) → B(x)
        m2 = re.match(r'^∀x: ([a-z_]+)\(x\) → ([a-z_]+)\(x\)$', ptxt)
        if m2:
            src = m2.group(1)
            tgt = m2.group(2)
            rules.append((src, tgt, ptxt))
        # fact: pred(const)
        m3 = re.match(r'^([a-z_]+)\((.+)\)$', ptxt)
        if m3 and not ptxt.startswith("∀x"):
            facts.append((m3.group(1), m3.group(2), ptxt))

    # Now see if some rule tgt == goal_pred and we have the src on the same const
    for (src, tgt, ptxt) in rules:
        if tgt == goal_pred:
            # do we have src(goal_const)?
            for (fpred, fconst, ftxt) in facts:
                if fpred == src and fconst == goal_const:
                    steps.append(f"1. From premise: `{ftxt}` we know `{src}({goal_const})`.")
                    steps.append(f"2. From universal: `{ptxt}` we know that whenever `{src}(x)` holds, `{tgt}(x)` holds.")
                    steps.append(f"3. By instantiating x = {goal_const}, we conclude `{tgt}({goal_const})`, i.e. **{goal_pretty}**.")
                    return steps

    # If we reach here, we didn't find a simple 2-step chain
    steps.append("No direct rule+fact chain found to derive this goal.")
    return steps

In [ ]:
# 10. Diagnostics for NOT ENTAILED

def explain_failure(goal_str, goal_preds, goal_consts, premise_summaries):
    reasons = []

    all_prem_consts = set()
    for ps in premise_summaries:
        all_prem_consts |= ps.get("consts", set())
    if goal_consts and not goal_consts.issubset(all_prem_consts):
        missing = goal_consts - all_prem_consts
        reasons.append(f"- Goal mentions constant(s) {', '.join(str(m) for m in missing)} but premises never introduce them.")

    all_prem_preds = set()
    for ps in premise_summaries:
        all_prem_preds |= ps.get("preds", set())
    if goal_preds and not goal_preds.issubset(all_prem_preds):
        missingp = goal_preds - all_prem_preds
        reasons.append(f"- Goal predicate(s) {', '.join(missingp)} never appear in premises.")

    if not reasons and goal_preds:
        reasons.append("- Premises do not provide a chain to derive this goal from existing facts.")

    if not reasons:
        reasons.append("- Z3 found a model where the goal is false, so it is not a logical consequence of the premises.")

    return "\n".join(reasons)

In [ ]:
# 11. Orchestrator (R08) — adds narrative

def run_pipeline(user_text: str):
    consts = {}
    prem_texts, goal_texts = parse_input_block(user_text)

    translated = []
    z3_premises = []
    premise_summaries = []
    notes = []

    # translate premises
    for p in prem_texts:
        tr = translate_sentence_to_z3(p, consts)
        translated.append(tr)
        if tr["fol"] is not None:
            z3_premises.append(tr["fol"])
        else:
            notes.append(tr["pretty"])

        p_consts = set(v for v in consts.values())
        premise_summaries.append({
            "raw": tr["raw"],
            "preds": tr.get("preds", set()),
            "consts": p_consts
        })

    # translate goals
    goal_forms = []
    for g in goal_texts:
        g_z3, g_pretty, g_preds, g_consts = goal_to_z3(g, consts)
        goal_forms.append((g, g_z3, g_pretty, g_preds, g_consts))

    reasoning_results = []
    for g_orig, g_z3, g_pretty, g_preds, g_consts in goal_forms:
        if g_z3 is None:
            reasoning_results.append({
                "goal": g_orig,
                "status": "unparsed",
                "message": g_pretty,
                "narrative": None
            })
            continue
        entailed, model = check_entailment(z3_premises, g_z3)
        if entailed:
            narrative = build_proof_narrative(g_pretty, translated)
            reasoning_results.append({
                "goal": g_orig,
                "status": "entailed",
                "explanation": f"{g_pretty} is entailed by refutation.",
                "narrative": narrative,
                "model": None
            })
        else:
            diag = explain_failure(g_pretty, g_preds, g_consts, premise_summaries)
            reasoning_results.append({
                "goal": g_orig,
                "status": "not_entailed",
                "explanation": f"{g_pretty} is not implied; premises ∧ ¬goal is SAT.",
                "diagnostic": diag,
                "narrative": None,
                "model": str(model)
            })

    # pretty print
    print("=== INPUT ===")
    print("Premises:", " | ".join(prem_texts) if prem_texts else "(none)")
    print("Goals:", " | ".join(goal_texts) if goal_texts else "(none)")

    print("\n=== TRANSLATIONS ===")
    for tr in translated:
        print(f"{tr['raw']}  -->  {tr['pretty']}")
    for n in notes:
        print(n)

    print("\n=== RESULTS ===")
    for rr in reasoning_results:
        if rr["status"] == "entailed":
            print(f"[OK] {rr['goal']} --> ENTAILED.")
            if rr["narrative"]:
                print("  Narrative:")
                for line in rr["narrative"]:
                    print("   ", line)
        else:
            print(f"[NO] {rr['goal']} --> NOT ENTAILED.")
            print("  Reason:", rr["explanation"])
            if rr.get("diagnostic"):
                print("  Diagnostics:")
                for line in rr["diagnostic"].split("\n"):
                    print("   ", line)
            print("  Countermodel:", rr["model"])

    print("\n=== PREDICATES (debug) ===")
    for (pname, ar), fn in PRED_CACHE.items():
        print(f"{pname}/{ar}")

    return {
        "premises": prem_texts,
        "goals": goal_texts,
        "translations": translated,
        "results": reasoning_results,
    }

In [ ]:
# 12. Demo — classic syllogism (should produce a 3-step narrative)
demo = """
Premises: All men are mortal. Alex is a man.
Goals: Alex is mortal.
"""
summary = run_pipeline(demo)

=== INPUT ===
Premises: All men are mortal | Alex is a man.
Goals: Alex is mortal.

=== TRANSLATIONS ===
All men are mortal  -->  ∀x: man(x) → mortal(x)
Alex is a man  -->  man(alex)

=== RESULTS ===
[OK] Alex is mortal. --> ENTAILED.
  Narrative:
    1. From premise: `man(alex)` we know `man(alex)`.
    2. From universal: `∀x: man(x) → mortal(x)` we know that whenever `man(x)` holds, `mortal(x)` holds.
    3. By instantiating x = alex, we conclude `mortal(alex)`, i.e. **mortal(alex)**.

=== PREDICATES (debug) ===
man/1
mortal/1


In [ ]:
# 13. Export MD (optional)
def build_md(summary):
    lines = []
    lines.append("# AGI Reasoning Pipeline — R08 (Proof Narrative)")
    lines.append("## Premises")
    for p in summary["premises"]:
        lines.append(f"- {p}")
    lines.append("## Goals")
    for g in summary["goals"]:
        lines.append(f"- {g}")
    lines.append("## Results")
    for r in summary["results"]:
        if r["status"] == "entailed":
            lines.append(f"- ✅ {r['goal']}")
            if r["narrative"]:
                lines.append("  - Narrative:")
                for line in r["narrative"]:
                    lines.append(f"    - {line}")
        else:
            lines.append(f"- ❌ {r['goal']}")
            lines.append(f"  - {r.get('diagnostic','')}")
    return "\n".join(lines)

md_text = build_md(summary)
with open("AGI_Reasoning_Pipeline_R08.md", "w") as f:
    f.write(md_text)
print("Exported to AGI_Reasoning_Pipeline_R08.md")

Exported to AGI_Reasoning_Pipeline_R08.md
